# Study large Hexapod motion failures
Carig Lage - 28-Jul-25

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.efdUtils import getEfdData
from lsst.summit.utils.utils import dayObsIntToString
from astropy.time import Time, TimeDelta
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
expId = 2025071600177
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

In [ ]:
hex_app = getEfdData(
    client=client,
    topic="lsst.sal.MTHexapod.application",
    columns=["*"],
    expRecord=expRecord
)
camhex_app = hex_app[hex_app["salIndex"] == 1]
m2hex_app = hex_app[hex_app["salIndex"] == 2]

comp_offset = getEfdData(
    client=client,
    topic="lsst.sal.MTHexapod.logevent_compensationOffset",
    columns=["*"],
    expRecord=expRecord
)
cam_comp_offset = comp_offset[comp_offset["salIndex"] == 1]
m2_comp_offset = comp_offset[comp_offset["salIndex"] == 2]

comp_position = getEfdData(
    client=client,
    topic="lsst.sal.MTHexapod.logevent_compensatedPosition",
    columns=["*"],
    expRecord=expRecord
)
cam_comp_position = comp_position[comp_position["salIndex"] == 1]
m2_comp_position = comp_position[comp_position["salIndex"] == 2]


In [ ]:
m2hex_app.columns

In [ ]:
expRecord.timespan.begin

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))
m2hex_app['position0'].plot(ax=ax, label='M2 X-Position')
m2hex_app['demand0'].plot(ax=ax, label='M2 X-Demand')
ax.set_title(f"{expId} M2 X hexapod motion")
ax.axvline(Time(expRecord.timespan.begin).utc.isot, ls='--', color='green', label='Exposure start')
ax.axvline(Time(expRecord.timespan.end).utc.isot, ls='--', color='red', label='Exposure end')
ax.set_ylabel("Position (micron)")
ax.legend()

In [ ]:
comp_offset